In [3]:
# Import required libraries
import pandas as pd
import sys
import os

# Add the masters directory to path to import database manager
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'masters'))

from database_manager import MasterTablesManager

# Initialize database manager and connect
db_manager = MasterTablesManager()
conn = db_manager.connect_to_database()

# Load NBA players master table
query = """
SELECT *
FROM nba_games 
ORDER BY createdat DESC
"""

nba_players_df = pd.read_sql_query(query, conn)
conn.close()

nba_players_df

Connected to RDS PostgreSQL database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_19804\481249774.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nba_players_df = pd.read_sql_query(query, conn)


,gameid,seasonid,gamedate,hometeamid,hometeamabbreviation,hometeamname,awayteamid,awayteamabbreviation,awayteamname,matchup,homewl,awaywl,seasontype,leaguename,createdat,updatedat
0,0012400074,002024-25,2024-10-18,1610612747,LAL,Los Angeles Lakers,1610612744,GSW,Golden State Warriors,GSW vs. LAL,L,W,Pre Season,NBA,2025-08-11 19:04:38.743188,2025-08-11 19:04:38.743188
1,0012400071,002024-25,2024-10-18,1610612763,MEM,Memphis Grizzlies,1610612748,MIA,Miami Heat,MIA @ MEM,L,W,Pre Season,NBA,2025-08-11 19:04:38.743188,2025-08-11 19:04:38.743188
2,0012400060,002024-25,2024-10-16,15025,ULM,Ratiopharm Ulm,1610612757,POR,Portland Trail Blazers,POR vs. ULM,L,W,Pre Season,NBA,2025-08-11 19:04:38.743188,2025-08-11 19:04:38.743188
3,0012400059,002024-25,2024-10-16,1610612750,MIN,Minnesota Timberwolves,1610612741,CHI,Chicago Bulls,CHI vs. MIN,L,W,Pre Season,NBA,2025-08-11 19:04:38.743188,2025-08-11 19:04:38.743188
4,0012400025,002024-25,2024-10-16,1610612737,ATL,Atlanta Hawks,1610612748,MIA,Miami Heat,MIA vs. ATL,L,W,Pre Season,NBA,2025-08-11 19:04:38.743188,2025-08-11 19:04:38.743188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52777,0022300017,002023-24,2023-11-14,1610612766,CHA,Charlotte Hornets,1610612748,MIA,Miami Heat,MIA @ CHA,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
52778,0022300021,002023-24,2023-11-14,1610612759,SAS,San Antonio Spurs,1610612760,OKC,Oklahoma City Thunder,OKC vs. SAS,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
52779,0022300023,002023-24,2023-11-14,1610612762,UTA,Utah Jazz,1610612757,POR,Portland Trail Blazers,POR @ UTA,W,L,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137
52780,0022300020,002023-24,2023-11-14,1610612753,ORL,Orlando Magic,1610612751,BKN,Brooklyn Nets,BKN vs. ORL,L,W,Regular Season,NBA,2025-08-11 18:52:21.891137,2025-08-11 18:52:21.891137


In [8]:
# Pull from CommonAllPlayers endpoint
from nba_api.stats.endpoints import commonallplayers

# Get all NBA players
all_players = commonallplayers.CommonAllPlayers(league_id='00',season='2011-12', is_only_current_season='0')
players_data = all_players.get_data_frames()[0]
players_data

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,76001,"Abdelnaby, Alaa",Alaa Abdelnaby,0,1990,1994,HISTADD_alaa_abdelnaby,alaa_abdelnaby,0,,,,,None,Y,00
1,76002,"Abdul-Aziz, Zaid",Zaid Abdul-Aziz,0,1968,1977,HISTADD_zaid_abdul-aziz,zaid_abdul-aziz,0,,,,,None,Y,00
2,76003,"Abdul-Jabbar, Kareem",Kareem Abdul-Jabbar,0,1969,1988,HISTADD_kareem_abdul-jabbar,kareem_abdul-jabbar,0,,,,,None,Y,00
3,51,"Abdul-Rauf, Mahmoud",Mahmoud Abdul-Rauf,0,1990,2000,mahmoud_abdul-rauf,mahmoud_abdul-rauf,0,,,,,None,Y,00
4,1505,"Abdul-Wahad, Tariq",Tariq Abdul-Wahad,0,1997,2003,tariq_abdul-wahad,tariq_abdul-wahad,0,,,,,None,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109,78650,"Zunic, Matt",Matt Zunic,0,1948,1948,HISTADD_matt_zunic,matt_zunic,0,,,,,None,Y,00
5110,1641783,"da Silva, Tristan",Tristan da Silva,0,2024,2025,tristan_da_silva,tristan_da_silva,0,,,,,None,Y,00
5111,1628427,"Čančar, Vlatko",Vlatko Čančar,0,2019,2025,vlatko_cancar,vlatko_čančar,0,,,,,None,Y,01
5112,1642365,"Đurišić, Nikola",Nikola Đurišić,0,2025,2025,nikola_durisic,nikola_đurišić,0,,,,,None,N,01


In [11]:
players_data[players_data['DISPLAY_FIRST_LAST']=='LeBron James']

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
2199,2544,"James, LeBron",LeBron James,1,2003,2025,lebron_james,lebron_james,1610612748,Miami,Heat,MIA,heat,heat,Y,00
